In [ ]:
# CELL 1 (do not change)
"""import statements and boiler plate code"""
import tensorflow_datasets as tfds
import tensorflow as tf 
import tensorflow.keras as ks
import tensorflow.keras.backend as K
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import PIL.Image
import pandas as pd
import pickle

from typing import *
import sys

if sys.version_info >= (3, 8, 0):
    from math import prod
else:
    # math.prod shim for Python 3.7 and older
    def prod(iterable, *, start=1):
        total = start
        for element in iterable:
            total *= element
        return (total)

def pretruberate(latent, model, bit, value):
    if not isinstance(latent, np.ndarray):
        latent = latent.numpy()
    latent[0][bit] = step
    return model.decode(latent)

def pretruberate_range(latent, image, model, bit, vals):
    fig, ax = plt.subplots(1, 11)
    fig.set_size_inches(w = 15, h = 30)
    for i, step in enumerate(vals):
        rec = pretruberate(latent, model, bit, step)
        ax[i].imshow(rec[0, :, :, 0])
        ax[i].axis('off')
    ax[10].imshow(image[0, :, :, 0])
    ax[10].axis('off')
    plt.show()
    pass

def get_sample(val, index, model, batch_size = None):
    sample = val.skip(index).take(1)
    for image, label in sample:
        break
    if batch_size is None:
        latent = model.predict(image).numpy()
    else:
        latent = model.predict(image, batch_size = batch_size).numpy()
    return image, latent


def display(val, model, n = 10, x0 = 0, y0 = 1): 
    image, latent = get_sample(val, 11, model)
    latent = model.predict(image).numpy()
    latent = latent[0, :]
    print(latent.shape)
    norm = tfp.distributions.Normal(0,1)
    vals_x = norm.quantile(np.linspace(0.05, 0.95, n))
    vals_y = norm.quantile(np.linspace(0.05, 0.95, n))

    fig, ax = plt.subplots(n, n)
    fig.set_size_inches(w = 10, h = 10)
    for i, x in enumerate(vals_x):
        for j, y in enumerate(vals_y):
            latent[x0], latent[y0] = x, y
            rec = tf.sigmoid(model.decode(np.array([latent])))
            ax[i, j].imshow(rec[0,:,:,0], cmap='gnuplot2')
            ax[i, j].axis('off')
    plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    plt.show()
    pass

In [ ]:
# CELL 2 preprocessing
'''
copy the pipeline above 

to the pipeline:
    - add a random flip
    - random hue, max alpha = 0.3
    - random brightness, max alpha = 0.3
    - random_contrast, min = 0.9, max = 1.1
'''
def preprocess(image: tf.Tensor, label: tf.Tensor, depth = 3, disp: bool = False):
    shape = tf.shape(image)
    image = tf.cast(image, dtype = tf.float32)
    image = image/255

    crop_1 = tf.cast(tf.shape(image)[0] * 22/28, dtype = tf.int32)
    crop_2 = tf.cast(tf.shape(image)[1] * 26/28, dtype = tf.int32)

    """
    use crop_1 and crop_2 as the range within which to generate a random number and random crop
    """
    randint = NotImplemented

    # new additions
    if (tf.shape(image)[-1] == 3):
        """
        add:
            - random flip
            - random hue, max alpha = 0.3
            - random brightness, max alpha = 0.1
            - random_contrast, min = 0.9, max = 1.1
        """

    image = tf.image.resize(image, (shape[0], shape[1]))
    label = tf.one_hot(label, depth)
    if disp:
        tf.print("shape: ", tf.shape(image))
        tf.print("random value: ", randint)
    return image, label

"""script to load data set and print info"""
(train, test), info = tfds.load("rock_paper_scissors", split = ['train', 'test'], with_info=True, shuffle_files=True, as_supervised = True)
print(info)

""" Test """
train = train.map(lambda x, y: preprocess(x, y, disp = True)).batch(1)
for i, (image, label) in enumerate(train):
    if (tf.shape(image)[-1] != 3):
        plt.imshow(image[0, ..., 0])
    else:
        plt.imshow(image[0, ...])
    plt.show()
    print(label)
    if i > 10:
        break

In [ ]:
# CELL 3 build model
model = ks.Sequential()
model.add(ks.layers.Conv2D("""fill this in"""))# inital feature extraction, get the most important information, so less filters is better, do a single down sample
"""

BUILD A Conv NN: 

ADD LAYERS TO BUILD A FULL CNN
    - add layers to the sequential model using model.add(<the layer you choose>)
JUST MAKE SURE YOU USE SOFTMAX AS THE LAST LAYER, the soft max layer is provided

competition:
    - the student with the lowest number of parameters and accuracy >= 90% on the validation set will win $15
    - if multiple people have the same number of parameters, the money will be split between the winners, or it will go to the student with higher accuracy.

"""
model.add(ks.layers.Activation(activation = "softmax"))

model.build(input_shape=(None, 300, 300, 3))
model.summary()

"""script to load data set and print info"""
with open('pokemon.pkl', 'rb') as f:
  df = pickle.load(f)                 # dataset dataframe
target = df.pop('Type1')

dataset = NotImplemented
(train, test) = NotImplemented

train = train.map(lambda x, y: preprocess(x, y, disp = False)).batch(40)
test = test.map(lambda x, y: preprocess(x, y, disp = False)).batch(40)

EPOCHS = 100
optimizer = NotImplemented
loss_fn = NotImplemented
metrics_fn = ks.metrics.CategoricalAccuracy()

"""script that will be used to test you model"""
model.compile(optimizer=optimizer, loss = loss_fn, metrics)
model.fit(train, validation_data = test, epochs=EPOCHS)